## d. Perform predictive analytics to understand the drivers of customers who are recommending the products.


### Classification Model

##### Vectorization (count, tfidf) for both train & test

In [8]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 
import seaborn as sns

In [9]:
df1 =pd.read_excel("Womens Clothing Reviews After Text mining.xlsx")
df1.head()

,Product_ID,Category,SubCategory1,SubCategory2,Location,Channel,Customer_Age,Rating,Recommend_Flag,Merged_Review,sentiment,sentiment_cat,age_group
0,767,Initmates,Intimate,Intimates,Mumbai,Mobile,33,4,1,Absolutely wonderful - silky and sexy and com...,0.633333,Positive,"(29.0, 39.0]"
1,1080,General,Dresses,Dresses,Bangalore,Mobile,34,5,1,Love this dress! it's sooo pretty. i happen...,0.339583,Positive,"(29.0, 39.0]"
2,1077,General,Dresses,Dresses,Gurgaon,Mobile,60,3,0,Some major design flaws I had such high hopes ...,0.073209,Negative,"(59.0, 69.0]"
3,1049,General Petite,Bottoms,Pants,Chennai,Web,50,5,1,"My favorite buy! I love, love, love this jumps...",0.560714,Positive,"(49.0, 59.0]"
4,847,General,Tops,Blouses,Bangalore,Web,47,5,1,Flattering shirt This shirt is very flattering...,0.512891,Positive,"(39.0, 49.0]"


#### Split the data in X and Y ( Recommend_Flag)


In [10]:
## X-variable is Review_text and y-variable is Rating
# define X and y
from sklearn.model_selection import train_test_split

X4 = df1.Merged_Review
y4 = df1.Recommend_Flag

# split the new DataFrame into training and testing sets
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, random_state=1)
print(X4_train.shape)
print(X4_test.shape)
print(y4_train.shape)
print(y4_test.shape)

(16981,)
(5661,)
(16981,)
(5661,)


In [12]:
X4_train = X4_train.apply(lambda x: clean_text(x))
X4_test = X4_test.apply(lambda x: clean_text(x))

In [16]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#Train
count_vect = CountVectorizer(analyzer='word', 
                             token_pattern=r'\w{1,}', 
                             ngram_range=(1, 1 ), 
                             min_df=5, 
                             lowercase = True,
                             encoding='latin-1' , 
                             max_features=100)
X_train_count4 = count_vect.fit_transform(X4_train)

tfidf_vect = TfidfVectorizer(analyzer='word', 
                             token_pattern=r'\w{1,}', 
                             ngram_range=(1, 1 ), 
                             min_df=5, 
                             encoding='latin-1' , 
                             lowercase = True,
                             max_features=100)
X_train_tfidf4 = tfidf_vect.fit_transform(X4_train)

#Test
X_test_count4 = count_vect.transform(X4_test)
X_test_tfidf4 = tfidf_vect.transform(X4_test)

In [18]:
# Convert to DataFrames with feature names

dtm_count = pd.DataFrame(X_train_count4.toarray(), columns=count_vect.get_feature_names_out())
dtm_tfidf = pd.DataFrame(X_train_tfidf4.toarray(), columns=tfidf_vect.get_feature_names_out())

#### Adding Features to a Document-Term Matrix


### Dummy Creation


In [19]:
df1.head()


,Product_ID,Category,SubCategory1,SubCategory2,Location,Channel,Customer_Age,Rating,Recommend_Flag,Merged_Review,sentiment,sentiment_cat,age_group
0,767,Initmates,Intimate,Intimates,Mumbai,Mobile,33,4,1,Absolutely wonderful - silky and sexy and com...,0.633333,Positive,"(29.0, 39.0]"
1,1080,General,Dresses,Dresses,Bangalore,Mobile,34,5,1,Love this dress! it's sooo pretty. i happen...,0.339583,Positive,"(29.0, 39.0]"
2,1077,General,Dresses,Dresses,Gurgaon,Mobile,60,3,0,Some major design flaws I had such high hopes ...,0.073209,Negative,"(59.0, 69.0]"
3,1049,General Petite,Bottoms,Pants,Chennai,Web,50,5,1,"My favorite buy! I love, love, love this jumps...",0.560714,Positive,"(49.0, 59.0]"
4,847,General,Tops,Blouses,Bangalore,Web,47,5,1,Flattering shirt This shirt is very flattering...,0.512891,Positive,"(39.0, 49.0]"


In [20]:
# An utility function to create dummy variable
def create_dummies(df, colname):
    col_dummies = pd.get_dummies(df[colname], prefix = colname, drop_first = True)
    df = pd.concat([df, col_dummies], axis = 1)
    df.drop(colname, axis = 1, inplace = True )
    return df

In [22]:
cat_vars = df1[['Category', 'SubCategory1', 'SubCategory2', 'sentiment_cat','Location', 'Channel']]

# for c_feature in categorical_features
for c_feature in ['Category', 'SubCategory1', 'SubCategory2', 'sentiment_cat','Location', 'Channel']:
    cat_vars[c_feature] = cat_vars[c_feature].astype('category')
    cat_vars = create_dummies(cat_vars, c_feature)
 
cat_vars.head()

C:\Users\Chichhorng Koem\AppData\Local\Temp\ipykernel_17924\500785668.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_vars[c_feature] = cat_vars[c_feature].astype('category')


,Category_General Petite,Category_Initmates,SubCategory1_Dresses,SubCategory1_Intimate,SubCategory1_Jackets,SubCategory1_Tops,SubCategory1_Trend,SubCategory2_Casual bottoms,SubCategory2_Chemises,SubCategory2_Dresses,...,SubCategory2_Sleep,SubCategory2_Sweaters,SubCategory2_Swim,SubCategory2_Trend,sentiment_cat_Nuetral,sentiment_cat_Positive,Location_Chennai,Location_Gurgaon,Location_Mumbai,Channel_Web
0,False,True,False,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
1,False,False,True,False,False,False,False,False,False,True,...,False,False,False,False,False,True,False,False,False,False
2,False,False,True,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,True,False,False,True
4,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True


In [23]:
dtm_count


,a,all,also,am,an,and,are,as,at,back,...,was,wear,well,when,which,will,with,work,would,you
0,3,0,0,0,1,4,0,0,0,0,...,1,1,1,0,2,0,0,0,0,0
1,2,0,1,0,1,2,1,0,0,1,...,1,0,0,1,2,0,0,0,0,0
2,1,0,0,0,0,6,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,3,0,0,1,0,3,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0
4,6,0,1,1,0,3,1,0,1,0,...,0,1,0,0,1,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16976,4,0,1,0,0,3,1,0,0,0,...,1,1,0,0,0,0,2,0,0,0
16977,5,0,0,0,1,3,1,0,0,1,...,0,1,1,0,1,1,1,0,0,1
16978,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16979,6,0,1,1,0,2,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [24]:
df1.head()


,Product_ID,Category,SubCategory1,SubCategory2,Location,Channel,Customer_Age,Rating,Recommend_Flag,Merged_Review,sentiment,sentiment_cat,age_group
0,767,Initmates,Intimate,Intimates,Mumbai,Mobile,33,4,1,Absolutely wonderful - silky and sexy and com...,0.633333,Positive,"(29.0, 39.0]"
1,1080,General,Dresses,Dresses,Bangalore,Mobile,34,5,1,Love this dress! it's sooo pretty. i happen...,0.339583,Positive,"(29.0, 39.0]"
2,1077,General,Dresses,Dresses,Gurgaon,Mobile,60,3,0,Some major design flaws I had such high hopes ...,0.073209,Negative,"(59.0, 69.0]"
3,1049,General Petite,Bottoms,Pants,Chennai,Web,50,5,1,"My favorite buy! I love, love, love this jumps...",0.560714,Positive,"(49.0, 59.0]"
4,847,General,Tops,Blouses,Bangalore,Web,47,5,1,Flattering shirt This shirt is very flattering...,0.512891,Positive,"(39.0, 49.0]"


In [26]:
df2 =df1.loc[:, [ 'Merged_Review', 'Customer_Age', 'Rating', 'sentiment','Recommend_Flag']]

df2.head()

,Merged_Review,Customer_Age,Rating,sentiment,Recommend_Flag
0,Absolutely wonderful - silky and sexy and com...,33,4,0.633333,1
1,Love this dress! it's sooo pretty. i happen...,34,5,0.339583,1
2,Some major design flaws I had such high hopes ...,60,3,0.073209,0
3,"My favorite buy! I love, love, love this jumps...",50,5,0.560714,1
4,Flattering shirt This shirt is very flattering...,47,5,0.512891,1


In [28]:
data = pd.concat([ df2,cat_vars], axis =1)
data.columns

Index(['Merged_Review', 'Customer_Age', 'Rating', 'sentiment',
       'Recommend_Flag', 'Category_General Petite', 'Category_Initmates',
       'SubCategory1_Dresses', 'SubCategory1_Intimate', 'SubCategory1_Jackets',
       'SubCategory1_Tops', 'SubCategory1_Trend',
       'SubCategory2_Casual bottoms', 'SubCategory2_Chemises',
       'SubCategory2_Dresses', 'SubCategory2_Fine gauge',
       'SubCategory2_Intimates', 'SubCategory2_Jackets', 'SubCategory2_Jeans',
       'SubCategory2_Knits', 'SubCategory2_Layering', 'SubCategory2_Legwear',
       'SubCategory2_Lounge', 'SubCategory2_Outerwear', 'SubCategory2_Pants',
       'SubCategory2_Shorts', 'SubCategory2_Skirts', 'SubCategory2_Sleep',
       'SubCategory2_Sweaters', 'SubCategory2_Swim', 'SubCategory2_Trend',
       'sentiment_cat_Nuetral', 'sentiment_cat_Positive', 'Location_Chennai',
       'Location_Gurgaon', 'Location_Mumbai', 'Channel_Web'],
      dtype='object')

In [29]:
data.head()


,Merged_Review,Customer_Age,Rating,sentiment,Recommend_Flag,Category_General Petite,Category_Initmates,SubCategory1_Dresses,SubCategory1_Intimate,SubCategory1_Jackets,...,SubCategory2_Sleep,SubCategory2_Sweaters,SubCategory2_Swim,SubCategory2_Trend,sentiment_cat_Nuetral,sentiment_cat_Positive,Location_Chennai,Location_Gurgaon,Location_Mumbai,Channel_Web
0,Absolutely wonderful - silky and sexy and com...,33,4,0.633333,1,False,True,False,True,False,...,False,False,False,False,False,True,False,False,True,False
1,Love this dress! it's sooo pretty. i happen...,34,5,0.339583,1,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
2,Some major design flaws I had such high hopes ...,60,3,0.073209,0,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
3,"My favorite buy! I love, love, love this jumps...",50,5,0.560714,1,True,False,False,False,False,...,False,False,False,False,False,True,True,False,False,True
4,Flattering shirt This shirt is very flattering...,47,5,0.512891,1,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True


In [32]:
#define X and y
feature_cols = [ 'Merged_Review', 'Customer_Age', 'Rating', 'sentiment','Recommend_Flag',
                'Category_General Petite', 'Category_Initmates','SubCategory1_Dresses', 'SubCategory1_Intimate',
                'SubCategory1_Jackets','SubCategory1_Tops', 'SubCategory1_Trend',
       'SubCategory2_Casual bottoms', 'SubCategory2_Chemises',
       'SubCategory2_Dresses', 'SubCategory2_Fine gauge',
       'SubCategory2_Intimates', 'SubCategory2_Jackets', 'SubCategory2_Jeans',
       'SubCategory2_Knits', 'SubCategory2_Layering', 'SubCategory2_Legwear',
       'SubCategory2_Lounge', 'SubCategory2_Outerwear', 'SubCategory2_Pants',
       'SubCategory2_Shorts', 'SubCategory2_Skirts', 'SubCategory2_Sleep',
       'SubCategory2_Sweaters', 'SubCategory2_Swim', 'SubCategory2_Trend',
       'sentiment_cat_Nuetral', 'sentiment_cat_Positive', 'Location_Chennai',
       'Location_Gurgaon', 'Location_Mumbai', 'Channel_Web']
X = data[feature_cols]
y = data.Recommend_Flag

#split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [34]:
X_train.Merged_Review


10853    Cute and comfy The tank top fits well. i usual...
7464     Itchy and odd fit I was so bummed when this ca...
17084    Basic everyday joggers I just purchased these ...
18045    I love the fit and length of this sweater! Thi...
19034    Great fit, perfect for summer I bought this cu...
                               ...                        
10955    Love these pants I bought these about a month ...
17289    Black is see-through up close I'm usually a sm...
5192     Runs small Should have ordered the xl instead ...
12172    Great Love these-although i'm exchanging mine ...
235      Recommend but not for me Love pilcro, love the...
Name: Merged_Review, Length: 16981, dtype: object

In [38]:
# use  TfidfVectorizer with Review_Text column only
vect = TfidfVectorizer(lowercase=True, stop_words='english', max_features=100, min_df=5, ngram_range=(1, 2))
X_train_dtm = vect.fit_transform(X_train.Merged_Review)
X_test_dtm = vect.transform(X_test.Merged_Review)
print(X_train_dtm.shape)
print(X_test_dtm.shape)

# shape of other four feature columns
X_train.drop('Merged_Review', axis=1).shape

(16981, 100)
(5661, 100)


(16981, 36)

In [40]:
print(tfidf_vect.get_feature_names_out())


['a' 'all' 'also' 'am' 'an' 'and' 'are' 'as' 'at' 'back' 'be' 'beautiful'
 'because' 'bit' 'bought' 'but' 'can' 'color' 'colors' 'comfortable'
 'cute' 'down' 'dress' 'even' 'fabric' 'fit' 'fits' 'flattering' 'for'
 'got' 'great' 'had' 'has' 'have' 'i' 'if' 'im' 'in' 'is' 'it' 'its'
 'jeans' 'just' 'large' 'length' 'like' 'little' 'long' 'look' 'looks'
 'love' 'material' 'me' 'medium' 'more' 'much' 'my' 'nice' 'not' 'of' 'on'
 'one' 'or' 'ordered' 'out' 'perfect' 'petite' 'pretty' 'quality' 'really'
 'shirt' 'size' 'small' 'so' 'soft' 'sweater' 'than' 'that' 'the' 'them'
 'these' 'they' 'think' 'this' 'to' 'too' 'top' 'up' 'very' 'waist' 'was'
 'wear' 'well' 'when' 'which' 'will' 'with' 'work' 'would' 'you']


In [42]:
# use CountVectorizer with Review_Text column only
vect = CountVectorizer(lowercase=True, stop_words='english', max_features=100, min_df=5, ngram_range=(1, 2))
X_train_dtm = vect.fit_transform(X_train.Merged_Review)
X_test_dtm = vect.transform(X_test.Merged_Review)
print(X_train_dtm.shape)
print(X_test_dtm.shape)

# shape of other four feature columns
X_train.drop('Merged_Review', axis=1).shape


(16981, 100)
(5661, 100)


(16981, 36)

In [44]:
print(vect.get_feature_names_out())


['_x000d_' 'beautiful' 'better' 'big' 'bit' 'black' 'blue' 'body' 'bought'
 'casual' 'color' 'colors' 'comfortable' 'comfy' 'cut' 'cute' 'definitely'
 'design' 'did' 'didn' 'does' 'don' 'dress' 'fabric' 'fall' 'feel' 'fit'
 'fits' 'flattering' 'going' 'good' 'gorgeous' 'got' 'great' 'high'
 'jacket' 'jeans' 'just' 'large' 'lbs' 'length' 'like' 'little' 'long'
 'look' 'looked' 'looking' 'looks' 'loose' 'love' 'loved' 'lovely'
 'material' 'medium' 'model' 'nice' 'online' 'ordered' 'pants' 'perfect'
 'perfectly' 'person' 'petite' 'piece' 'pretty' 'price' 'purchased'
 'quality' 'really' 'retailer' 'right' 'runs' 'sale' 'shirt' 'short'
 'size' 'skirt' 'sleeves' 'small' 'soft' 'store' 'style' 'summer' 'super'
 'sweater' 'think' 'tight' 'tried' 'true' 'true size' 'try' 'usually'
 'waist' 'wanted' 'way' 'wear' 'wearing' 'white' 'work' 'xs']


In [53]:
from scipy import sparse

# cast other feature columns to float and convert to a sparse matrix
extra = sparse.csr_matrix(X_train.drop('Merged_Review', axis=1).astype(float))
extra.shape

# combine sparse matrices
X_train_dtm_extra = sparse.hstack((X_train_dtm, extra))
X_train_dtm_extra.shape

# repeat for testing set
extra = sparse.csr_matrix(X_test.drop('Merged_Review', axis=1).astype(float))
X_test_dtm_extra = sparse.hstack((X_test_dtm, extra))
X_test_dtm_extra.shape

(5661, 136)

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# use logistic regression with text column only
logreg = LogisticRegression(C=1e9)
logreg.fit(X_train_dtm, y_train)
y_pred_class = logreg.predict(X_test_dtm)
print(metrics.accuracy_score(y_test, y_pred_class))

0.8572690337396219


In [58]:
print(dir(logreg))


['C', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__sklearn_clone__', '__str__', '__subclasshook__', '__weakref__', '_build_request_for_signature', '_check_feature_names', '_check_n_features', '_estimator_type', '_get_default_requests', '_get_metadata_request', '_get_param_names', '_get_tags', '_more_tags', '_parameter_constraints', '_predict_proba_lr', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_validate_data', '_validate_params', 'class_weight', 'classes_', 'coef_', 'decision_function', 'densify', 'dual', 'fit', 'fit_intercept', 'get_metadata_routing', 'get_params', 'intercept_', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_features_in_', 'n_iter_', 'n

In [59]:
# Finding the score for validation

from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import roc_auc_score,accuracy_score

tr_pred=logreg.predict(X_train_dtm)
y_pred = logreg.predict(X_test_dtm)


trprecision,trrecall,trfscore,trsupport=score(y_train,tr_pred)
tracc=accuracy_score(y_train,tr_pred)
precision,recall,fscore,support=score(y_test,y_pred)
acc=accuracy_score(y_test,y_pred)

In [60]:
# For Training

print('Precision : ',trprecision)
print('\nRecall : ',trrecall)
print('\nF-Score :',trfscore)
print('\nAccuracy : ',tracc)

Precision :  [0.67402096 0.8771097 ]

Recall :  [0.39598185 0.95746671]

F-Score : [0.49887732 0.91552833]

Accuracy :  0.8554266533184147


In [61]:
# For Testing

print('Precision : ',precision)
print('\nRecall : ',recall)
print('\nF-Score :',fscore)
print('\nAccuracy : ',acc)

Precision :  [0.67814114 0.87775591]

Recall :  [0.38817734 0.95975032]

F-Score : [0.49373434 0.91692371]

Accuracy :  0.8572690337396219


In [62]:
# use logistic regression with all features
#logreg1 = LogisticRegression(C=1e9)
#logreg1.fit(X_train_dtm_extra, y_train)
#y_pred_class = logreg.predict(X_test_dtm_extra)
#print(metrics.accuracy_score(y_test, y_pred_class))
#Saving model
import pickle
Pkl_Filename = "Pickle_LR_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(logreg, file)

In [63]:
# Load the Model back from file
with open(Pkl_Filename, 'rb') as file:  
    Pickled_LR_Model = pickle.load(file)

Pickled_LR_Model

LogisticRegression(C=1000000000.0)

In [64]:
# Use the Reloaded Model to 
# Calculate the accuracy score and predict target values

# Calculate the Score 
score = Pickled_LR_Model.score(X_test_dtm, y_test)  
# Print the Score
print("Test score: {0:.2f} %".format(100 * score))  

# Predict the Labels using the reloaded Model
Ypredict = Pickled_LR_Model.predict(X_test_dtm)  
Ypredict

Test score: 85.73 %


array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [67]:
print(vect.get_feature_names_out())


['_x000d_' 'beautiful' 'better' 'big' 'bit' 'black' 'blue' 'body' 'bought'
 'casual' 'color' 'colors' 'comfortable' 'comfy' 'cut' 'cute' 'definitely'
 'design' 'did' 'didn' 'does' 'don' 'dress' 'fabric' 'fall' 'feel' 'fit'
 'fits' 'flattering' 'going' 'good' 'gorgeous' 'got' 'great' 'high'
 'jacket' 'jeans' 'just' 'large' 'lbs' 'length' 'like' 'little' 'long'
 'look' 'looked' 'looking' 'looks' 'loose' 'love' 'loved' 'lovely'
 'material' 'medium' 'model' 'nice' 'online' 'ordered' 'pants' 'perfect'
 'perfectly' 'person' 'petite' 'piece' 'pretty' 'price' 'purchased'
 'quality' 'really' 'retailer' 'right' 'runs' 'sale' 'shirt' 'short'
 'size' 'skirt' 'sleeves' 'small' 'soft' 'store' 'style' 'summer' 'super'
 'sweater' 'think' 'tight' 'tried' 'true' 'true size' 'try' 'usually'
 'waist' 'wanted' 'way' 'wear' 'wearing' 'white' 'work' 'xs']


### So, Above we made one logistic model where Y variable is recommend_flag and X variable is review_text. The model gives accuracy of 84% in train and test. The difference between train and test accuracy is less so, we can say the model is good to use. The main key drivers who are responsible for recommending the product are as above.


In [69]:
#define X and y
feature_cols = [ 'Merged_Review', 'Customer_Age', 'sentiment','Recommend_Flag',
                'Category_General Petite', 'Category_Initmates','SubCategory1_Dresses', 'SubCategory1_Intimate',
                'SubCategory1_Jackets','SubCategory1_Tops', 'SubCategory1_Trend',
       'SubCategory2_Casual bottoms', 'SubCategory2_Chemises',
       'SubCategory2_Dresses', 'SubCategory2_Fine gauge',
       'SubCategory2_Intimates', 'SubCategory2_Jackets', 'SubCategory2_Jeans',
       'SubCategory2_Knits', 'SubCategory2_Layering', 'SubCategory2_Legwear',
       'SubCategory2_Lounge', 'SubCategory2_Outerwear', 'SubCategory2_Pants',
       'SubCategory2_Shorts', 'SubCategory2_Skirts', 'SubCategory2_Sleep',
       'SubCategory2_Sweaters', 'SubCategory2_Swim', 'SubCategory2_Trend',
       'sentiment_cat_Nuetral', 'sentiment_cat_Positive', 'Location_Chennai',
       'Location_Gurgaon', 'Location_Mumbai', 'Channel_Web']
X = data[feature_cols]
y = data.Rating

#split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [71]:
# use  TfidfVectorizer with Review_Text column only
vect = TfidfVectorizer(lowercase=True, stop_words='english', max_features=100, min_df=5, ngram_range=(1, 2))
X_train_dtm = vect.fit_transform(X_train.Merged_Review)
X_test_dtm = vect.transform(X_test.Merged_Review)
print(X_train_dtm.shape)
print(X_test_dtm.shape)

# shape of other four feature columns
X_train.drop('Merged_Review', axis=1).shape

(16981, 100)
(5661, 100)


(16981, 35)

In [73]:
print(vect.get_feature_names_out())


['_x000d_' 'beautiful' 'better' 'big' 'bit' 'black' 'blue' 'body' 'bought'
 'casual' 'color' 'colors' 'comfortable' 'comfy' 'cut' 'cute' 'definitely'
 'design' 'did' 'didn' 'does' 'don' 'dress' 'fabric' 'fall' 'feel' 'fit'
 'fits' 'flattering' 'going' 'good' 'gorgeous' 'got' 'great' 'high'
 'jacket' 'jeans' 'just' 'large' 'lbs' 'length' 'like' 'little' 'long'
 'look' 'looked' 'looking' 'looks' 'loose' 'love' 'loved' 'lovely'
 'material' 'medium' 'model' 'nice' 'online' 'ordered' 'pants' 'perfect'
 'perfectly' 'person' 'petite' 'piece' 'pretty' 'price' 'purchased'
 'quality' 'really' 'retailer' 'right' 'runs' 'sale' 'shirt' 'short'
 'size' 'skirt' 'sleeves' 'small' 'soft' 'store' 'style' 'summer' 'super'
 'sweater' 'think' 'tight' 'tried' 'true' 'true size' 'try' 'usually'
 'waist' 'wanted' 'way' 'wear' 'wearing' 'white' 'work' 'xs']


In [74]:
# use logistic regression with text column only
logreg2 = LogisticRegression(C=1e9)
logreg2.fit(X_train_dtm, y_train)
y_pred_class = logreg2.predict(X_test_dtm)
print(metrics.accuracy_score(y_test, y_pred_class))

0.5991874227168345


C:\Users\Chichhorng Koem\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [75]:
#Using KNN model
from sklearn.neighbors import KNeighborsClassifier

model3=KNeighborsClassifier(n_neighbors=5,n_jobs=-1)
model3.fit(X_train_dtm,y_train)

KNeighborsClassifier(n_jobs=-1)

In [76]:
# Finding the score for validation

from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import roc_auc_score,accuracy_score

tr_pred=logreg2.predict(X_train_dtm)
y_pred = logreg2.predict(X_test_dtm)


trprecision,trrecall,trfscore,trsupport=score(y_train,tr_pred)
tracc=accuracy_score(y_train,tr_pred)
precision,recall,fscore,support=score(y_test,y_pred)
acc=accuracy_score(y_test,y_pred)

In [77]:
# For Training

print('Precision : ',trprecision)
print('\nRecall : ',trrecall)
print('\nF-Score :',trfscore)
print('\nAccuracy : ',tracc)

Precision :  [0.3649635  0.32454361 0.38529718 0.43855816 0.68327316]

Recall :  [0.16207455 0.13793103 0.34776471 0.2179696  0.90654604]

F-Score : [0.22446689 0.19358742 0.36557012 0.2912058  0.77923147]

Accuracy :  0.6076791708379954


In [78]:
# For Testing

print('Precision : ',precision)
print('\nRecall : ',recall)
print('\nF-Score :',fscore)
print('\nAccuracy : ',acc)

Precision :  [0.34883721 0.3030303  0.33552632 0.41826923 0.68142652]

Recall :  [0.14705882 0.1285347  0.29226361 0.21323529 0.90495868]

F-Score : [0.20689655 0.18050542 0.31240429 0.28246753 0.77744402]

Accuracy :  0.5991874227168345


### Create user defined function for train Classification the models


In [79]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid,  valid_y, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [81]:
#Naive Bayes (With only review_text in X-vribles)
# Naive Bayes on Count Vectors and TF-IDF
from sklearn import naive_bayes

accuracy_L1 = train_model(naive_bayes.MultinomialNB(), X_train_dtm, y_train, X_test_dtm, y_test)
print("NB  for L1, TFIDF Vectors: ", accuracy_L1)

NB  for L1, TFIDF Vectors:  0.5594417947359124


In [82]:
# Naive Bayes on Word Level TF IDF Vectors
#accuracy_L1 = train_model(naive_bayes.MultinomialNB(), X_train_count, y_train, X_test_count, y_test)#
#print("NB  for L1, WordLevel Count: ", accuracy_L1)
#Logistic Regression
# Logistic Regression on Count Vectors and TF-IDF
accuracy_L1 = train_model(LogisticRegression(), X_train_dtm, y_train, X_test_dtm, y_test)
print("LR  for L1, tfidf Vectors: ", accuracy_L1)

# Logistic Regression on Word Level TF IDF Vectors
#accuracy_L1 = train_model(LogisticRegression(), X_train_count, y_train, X_test_count, y_test)
#print("LR  for L1, WordLevel count: ", accuracy_L1)

LR  for L1, tfidf Vectors:  0.600247306129659


C:\Users\Chichhorng Koem\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [84]:
#Linear SVC
# Linear SVC on Count Vectors and TF-IDF
from sklearn import svm

accuracy_L1 = train_model(svm.LinearSVC(), X_train_dtm, y_train, X_test_dtm, y_test)
print("SVC  for L1, Count Vectors: ", accuracy_L1)

# Linear SVC on Word Level TF IDF Vectors
#accuracy_L1 = train_model(svm.LinearSVC(), X_train_count, y_train, X_test_count, y_test)
#print("SVC  for L1, WordLevel TF-IDF: ", accuracy_L1)

C:\Users\Chichhorng Koem\AppData\Roaming\Python\Python311\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


SVC  for L1, Count Vectors:  0.5968910086557145


#### Hence from above logistic regression gives best result.save the model as pickle object.


In [86]:
print(vect.get_feature_names_out())


['_x000d_' 'beautiful' 'better' 'big' 'bit' 'black' 'blue' 'body' 'bought'
 'casual' 'color' 'colors' 'comfortable' 'comfy' 'cut' 'cute' 'definitely'
 'design' 'did' 'didn' 'does' 'don' 'dress' 'fabric' 'fall' 'feel' 'fit'
 'fits' 'flattering' 'going' 'good' 'gorgeous' 'got' 'great' 'high'
 'jacket' 'jeans' 'just' 'large' 'lbs' 'length' 'like' 'little' 'long'
 'look' 'looked' 'looking' 'looks' 'loose' 'love' 'loved' 'lovely'
 'material' 'medium' 'model' 'nice' 'online' 'ordered' 'pants' 'perfect'
 'perfectly' 'person' 'petite' 'piece' 'pretty' 'price' 'purchased'
 'quality' 'really' 'retailer' 'right' 'runs' 'sale' 'shirt' 'short'
 'size' 'skirt' 'sleeves' 'small' 'soft' 'store' 'style' 'summer' 'super'
 'sweater' 'think' 'tight' 'tried' 'true' 'true size' 'try' 'usually'
 'waist' 'wanted' 'way' 'wear' 'wearing' 'white' 'work' 'xs']


##### Above are main key drivers of customers who are recommending the products.
